# 如何用sklearn計算中文文本TF-IDF
2019-02-16

## 2.1 語料集
本文所用語料集為人機對話系統中的短文本語料，corpus列表中的每個元素是一條Query。（如果是长文本的话，每个元素是一篇文档）

In [1]:
corpus = [
  "幫我 查下 明天 北京 天氣 怎麼樣",
  "幫我 查下 今天 北京 天氣 好不好",
  "幫我 查詢 去 北京 的 火車",
  "幫我 查看 到 上海 的 火車",
  "幫我 查看 特朗普 的 新聞",
  "幫我 看看 有沒有 北京 的 新聞",
  "幫我 搜索 上海 有 什麼 好玩的",
  "幫我 找找 上海 東方明珠 在哪"
]

## 2.2 將語料轉換為詞袋向量

### step 1. 聲明一個向量化工具vectorizer

本文使用的是CountVectorizer，默認情況下，CountVectorizer僅統計長度超過兩個字符的詞，但是在短文本中任何一個字都可能十分重要，比如“去／到”等，所以要想讓CountVectorizer也支持單字符的詞，需要加上參數token_pattern='\\b\\w+\\b'。

### step 2.根據語料集統計詞袋（fit）；

### step 3.打印語料集的詞袋信息；

### step 4.將語料集轉化為詞袋向量（transform）；

### step 5.還可以查看每個詞在詞袋中的索引；

### step 6.將詞袋向量轉化為 numpy array，方便之後比較

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# step 1
vectorizer = CountVectorizer(min_df=1, max_df=1.0, token_pattern='\\b\\w+\\b')

# step 2
vectorizer.fit(corpus)

# step 3
bag_of_words = vectorizer.get_feature_names()
print("Bag of words:")
print(bag_of_words)
print(len(bag_of_words))

# step 4
X = vectorizer.transform(corpus)
print("Vectorized corpus:")
print(X.toarray())

# Step 4.5
# Step 3&4 can be combined into Step 4.5
# fit + transform = fit_transform
#Y = vectorizer.fit_transform(corpus)

# step 5
print("index of `什麼` is : {}".format(vectorizer.vocabulary_.get('什麼')))

# step 6
corpus_vecs = X.toarray()

Bag of words:
['上海', '什麼', '今天', '到', '北京', '去', '在哪', '天氣', '好不好', '好玩的', '幫我', '怎麼樣', '找找', '搜索', '新聞', '明天', '有', '有沒有', '東方明珠', '查下', '查看', '查詢', '火車', '特朗普', '的', '看看']
26
Vectorized corpus:
[[0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]]
index of `什麼` is : 1


In [3]:
X

<8x26 sparse matrix of type '<class 'numpy.int64'>'
	with 46 stored elements in Compressed Sparse Row format>

In [4]:
print(X)

  (0, 4)	1
  (0, 7)	1
  (0, 10)	1
  (0, 11)	1
  (0, 15)	1
  (0, 19)	1
  (1, 2)	1
  (1, 4)	1
  (1, 7)	1
  (1, 8)	1
  (1, 10)	1
  (1, 19)	1
  (2, 4)	1
  (2, 5)	1
  (2, 10)	1
  (2, 21)	1
  (2, 22)	1
  (2, 24)	1
  (3, 0)	1
  (3, 3)	1
  (3, 10)	1
  (3, 20)	1
  (3, 22)	1
  (3, 24)	1
  (4, 10)	1
  (4, 14)	1
  (4, 20)	1
  (4, 23)	1
  (4, 24)	1
  (5, 4)	1
  (5, 10)	1
  (5, 14)	1
  (5, 17)	1
  (5, 24)	1
  (5, 25)	1
  (6, 0)	1
  (6, 1)	1
  (6, 9)	1
  (6, 10)	1
  (6, 13)	1
  (6, 16)	1
  (7, 0)	1
  (7, 6)	1
  (7, 10)	1
  (7, 12)	1
  (7, 18)	1


In [5]:
type(corpus_vecs)

numpy.ndarray

## 2.3 根據詞袋向量統計TF-IDF
### step 1.聲明一個TF-IDF轉化器（TfidfTransformer）；
### step 2.根據語料集的詞袋向量計算TF-IDF（fit）；
### step 3.打印TF-IDF信息：比如結合詞袋信息，可以查看每個詞的TF-IDF值；
### step 4.將語料集的詞袋向量表示轉換為TF-IDF向量表示；

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

# step 1
tfidf_transformer = TfidfTransformer()

# step 2
tfidf_transformer.fit(X.toarray())

# step 3
for idx, word in enumerate(vectorizer.get_feature_names()):
    print("{}\t{}".format(word, tfidf_transformer.idf_[idx]))

# step 4
tfidf = tfidf_transformer.transform(X)
print(tfidf.toarray())

上海	1.8109302162163288
什麼	2.504077396776274
今天	2.504077396776274
到	2.504077396776274
北京	1.587786664902119
去	2.504077396776274
在哪	2.504077396776274
天氣	2.09861228866811
好不好	2.504077396776274
好玩的	2.504077396776274
幫我	1.0
怎麼樣	2.504077396776274
找找	2.504077396776274
搜索	2.504077396776274
新聞	2.09861228866811
明天	2.504077396776274
有	2.504077396776274
有沒有	2.504077396776274
東方明珠	2.504077396776274
查下	2.09861228866811
查看	2.09861228866811
查詢	2.504077396776274
火車	2.09861228866811
特朗普	2.504077396776274
的	1.587786664902119
看看	2.504077396776274
[[0.         0.         0.         0.         0.3183848  0.
  0.         0.42081614 0.         0.         0.20052115 0.50212047
  0.         0.         0.         0.50212047 0.         0.
  0.         0.42081614 0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.50212047 0.         0.3183848  0.
  0.         0.42081614 0.50212047 0.         0.20052115 0.
  0.         0.         0.         0.         0.         0.
  0.         0.4

In [7]:
tfidf

<8x26 sparse matrix of type '<class 'numpy.float64'>'
	with 46 stored elements in Compressed Sparse Row format>

In [8]:
print(tfidf)

  (0, 19)	0.4208161427072783
  (0, 15)	0.502120471152222
  (0, 11)	0.502120471152222
  (0, 10)	0.2005211467499556
  (0, 7)	0.4208161427072783
  (0, 4)	0.3183848028404604
  (1, 19)	0.4208161427072783
  (1, 10)	0.2005211467499556
  (1, 8)	0.502120471152222
  (1, 7)	0.4208161427072783
  (1, 4)	0.3183848028404604
  (1, 2)	0.502120471152222
  (2, 24)	0.3311691914996199
  (2, 22)	0.4377135482191675
  (2, 21)	0.5222825618037251
  (2, 10)	0.208572851013354
  (2, 5)	0.5222825618037251
  (2, 4)	0.3311691914996199
  (3, 24)	0.33944982039617033
  (3, 22)	0.44865823615762696
  (3, 20)	0.44865823615762696
  (3, 10)	0.2137880534581112
  (3, 3)	0.535341832365254
  (3, 0)	0.38715524587336536
  (4, 24)	0.3681610266939738
  (4, 23)	0.5806218969442964
  (4, 20)	0.48660646414754405
  (4, 14)	0.48660646414754405
  (4, 10)	0.23187058742344932
  (5, 25)	0.5222825618037251
  (5, 24)	0.3311691914996199
  (5, 17)	0.5222825618037251
  (5, 14)	0.4377135482191675
  (5, 10)	0.208572851013354
  (5, 4)	0.3311691914996

在(index1,index2)中：index1表示為第幾個句子或者文檔，index2為所有語料庫中的單詞組成的詞典的序號，之後的數字為該詞所計算得到的TF－idf的結果值

## 2.3.1 TfidfVectorizer

TfidfTransformer和TfidfVectorizer
具體來說，TfidfTransformer的作用已經不是在特徵提取了，而是特徵加權。而文本特徵權重的計算方法有許多，scikit-learn貌似也只提供了TF-TDF這一種。

關於TfidfTransformer給出的例程尤其簡單，這裡就不進行講述。

而TfidfVectorizer則是CountVectorizer和TfidfTransformer的結合版本。

一般在文本特徵處理過程中，本來正常的流程就是先用CountVectorizer來提取特徵，然後就用TfidfTransformer來計算特徵的權重。而TfidfVectorizer則是把兩者的功能合在一起，連參數也都是兩者的參數合在一起，所以可以方便的直接使用TfidfVectorizer。但是如果想在CountVectorizer來提取特徵後想處理特徵，比如降維之類的，這樣直接使用TfidfVectorizer就不行了。

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# step 1
vectorizer = TfidfVectorizer(sublinear_tf=True)
#vectorizer = TfidfVectorizer(stop_words=stpwrdlst, sublinear_tf=True, max_df=0.5)

'''
# step 2
vectorizer.fit(corpus)

# step 3
bag_of_words = vectorizer.get_feature_names()
print("Bag of words:")
print(bag_of_words)
print(len(bag_of_words))

# step 4
X = vectorizer.transform(corpus)
print("Vectorized corpus:")
print(X.toarray())
'''

# Step 4.5
# Step 2&3&4 can be combined into Step 4.5
# fit + transform = fit_transform
X = vectorizer.fit_transform(corpus)

bag_of_words = vectorizer.get_feature_names()
print("Bag of words:")
print(bag_of_words)
print(len(bag_of_words))
print("Vectorized corpus:")
print(X.toarray())

# step 5
print("index of `什麼` is : {}".format(vectorizer.vocabulary_.get('什麼')))

# step 6
corpus_vecs = X.toarray()

Bag of words:
['上海', '什麼', '今天', '北京', '在哪', '天氣', '好不好', '好玩的', '幫我', '怎麼樣', '找找', '搜索', '新聞', '明天', '有沒有', '東方明珠', '查下', '查看', '查詢', '火車', '特朗普', '看看']
22
Vectorized corpus:
[[0.         0.         0.         0.3183848  0.         0.42081614
  0.         0.         0.20052115 0.50212047 0.         0.
  0.         0.50212047 0.         0.         0.42081614 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.50212047 0.3183848  0.         0.42081614
  0.50212047 0.         0.20052115 0.         0.         0.
  0.         0.         0.         0.         0.42081614 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.42141948 0.         0.
  0.         0.         0.26541316 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.6646151  0.55699932 0.         0.        ]
 [0.50057382 0.         0.         0.         0.         0.
  0.         0.         0.27641806 0.         0.         0.
  0.   

In [10]:
X

<8x22 sparse matrix of type '<class 'numpy.float64'>'
	with 39 stored elements in Compressed Sparse Row format>

In [11]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### TfidfTransformer和TfidfVectorizer

具體來說，TfidfTransformer的作用已經不是在特徵提取了，而是特徵加權。而文本特徵權重的計算方法有許多，scikit-learn貌似也只提供了TF-TDF這一種。

關於TfidfTransformer給出的例程尤其簡單，這裡就不進行講述。

而TfidfVectorizer則是CountVectorizer和TfidfTransformer的結合版本。

一般在文本特徵處理過程中，本來正常的流程就是先用CountVectorizer來提取特徵，然後就用TfidfTransformer來計算特徵的權重。而TfidfVectorizer則是把兩者的功能合在一起，連參數也都是兩者的參數合在一起，所以可以方便的直接使用TfidfVectorizer。但是如果想在CountVectorizer來提取特徵後想處理特徵，比如降維之類的，這樣直接使用TfidfVectorizer就不行了。

## 2.4 儲存 TFIDF Model

In [ ]:
'''https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

# tf-idf based vectors
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True, max_features = 500000)

# Fit the model
tf_transformer = tf.fit(corpus)

# Dump the file
pickle.dump(tf_transformer, open("tfidf1.pkl", "wb"))


# Testing phase
tf1 = pickle.load(open("tfidf1.pkl", 'rb'))

# Create new tfidfVectorizer with old vocabulary
tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True,
                          max_features = 500000, vocabulary = tf1.vocabulary_)
X_tf1 = tf1_new.fit_transform(new_corpus)
'''

In [13]:
from sklearn.externals import joblib

# save your model in disk
joblib.dump(vectorizer, 'data/tfidf/tfidf.pkl')

['data/tfidf/tfidf.pkl']

In [14]:
# load your model
vectorizer = joblib.load('data/tfidf/tfidf.pkl')

In [15]:
query = "我 想 查 去 北京 的 車票"

vectorizer.transform([query])[0]

<1x22 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

## 2.5 計算相似度 Similarity

In [16]:
import numpy as np
def get_sim_paragraphs(query_vec, paragraph_vecs, limit=5):
    ''' Get top X paragrahs that similar to query
        input: narray
        output: 
    '''
    match_scores = []; match_paragraphs = []
    
    # compute simple dot product as score
    scores = np.sum(query_vec * paragraph_vecs, axis=1)
    topk_idx = np.argsort(scores)[::-1][:limit]
    for idx in topk_idx:
        print('> %s\t%s\t%s' % (idx, scores[idx], corpus[idx]))
        match_scores.append(scores[idx])
        match_paragraphs.append(corpus[idx])
    return match_scores, match_paragraphs

In [17]:
query = "我 想 查 去 北京 的 車票"
query_vec = vectorizer.transform([query])[0]
print(type(query_vec))

query_vec = query_vec.toarray()
print(type(query_vec))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [18]:
%%time
match_scores, match_paragraphs = get_sim_paragraphs(query_vec, corpus_vecs)

> 2	0.42141947906854116	幫我 查詢 去 北京 的 火車
> 5	0.3509741767018308	幫我 看看 有沒有 北京 的 新聞
> 1	0.3183848028404604	幫我 查下 今天 北京 天氣 好不好
> 0	0.3183848028404604	幫我 查下 明天 北京 天氣 怎麼樣
> 7	0.0	幫我 找找 上海 東方明珠 在哪
CPU times: user 698 µs, sys: 576 µs, total: 1.27 ms
Wall time: 694 µs


In [19]:
print(match_scores)
print(match_paragraphs)

[0.42141947906854116, 0.3509741767018308, 0.3183848028404604, 0.3183848028404604, 0.0]
['幫我 查詢 去 北京 的 火車', '幫我 看看 有沒有 北京 的 新聞', '幫我 查下 今天 北京 天氣 好不好', '幫我 查下 明天 北京 天氣 怎麼樣', '幫我 找找 上海 東方明珠 在哪']


https://my.oschina.net/u/2293326/blog/1838918